In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn import tree

# Decision Tree 

In [2]:
class Node(object):
    def __init__(self, name=None, value=None, delta=None, gini_after=None):
        self.feature_name = name
        self.feature_value = value
        self.delta = delta
        self.gini_after = gini_after
        self.left = None
        self.right = None

In [3]:
class DecisionTree(object):
    def __init__(self, depth=2, score='gini'):
        """
        param depth: speicifies depth of the three. Root node is depth 0
        param score: speicifes score to use for split
        """
        self._depth = depth
        self._score = score
        self._current_depth = -1
        self._tree = []
    def fit(self, X_train, y_train):
        """
        recursively builds the tree. At each level, 
        it finds the best node, 
        which is calculated by examining all the unique
        values of all the features
        and calculates delta value. 
        the feature-value pair with highest delta 
        value is chosen for split.
        Delta value is calculated by subtracting 
        the Gini index of data the 
        function has received, from 
        the Gini index after for a
        every feature-value pair. 
        """
        X_train.index = range(len(X_train.index))
        y_train.index = range(len(y_train.index))
        if self._current_depth == self._depth:
            node = Node(name='leaf', value=y_train)
            self._tree.append(node)
            print('class distribution: ', y_train)
            return node
        self._current_depth += 1
        print('cur_depth: ', self._current_depth)
        features = X_train.columns.values
        node = self._best_node(X_train, y_train)
        if not node:
            node = Node(name='leaf', value=y_train)
            self._tree.append(node)
            print('this is a pure leaf')
            print('class distribution: ', y_train)
            self._current_depth -= 1
            return node
        print('feature_name: ', node.feature_name,
              'feature_split: ', node.feature_value,
              'delta: ', node.delta, 
             'gini: ', node.gini_after)
        left_subset_X, left_subset_y, right_subset_X, right_subset_y = self._subset(node, X_train, y_train)
        node.left = self.fit(left_subset_X, left_subset_y)
        node.right = self.fit(right_subset_X, right_subset_y)
        self._tree.append(node)
        self._current_depth -= 1
        return node
    def _tree_search(self, data_point, node):
        """
        This method is a recursive solution to find the correct 
        leaf node for a test data_point.
        """
        if node.feature_name == 'leaf':
            return node.feature_value         
        elif data_point[node.feature_name] > node.feature_value:
            cls_dist = self._tree_search(data_point, node.right)
        elif data_point[node.feature_name] <= node.feature_value:
            cls_dist = self._tree_search(data_point, node.left)
        return cls_dist

    def predict_proba(self, data):
        """
        Gets the class distribution in the leaf where the new data point
        belongs and calculates mode of that
        distribution to calculate the class,
        and also returns probability.
        """
        label = []
        prob = []
        for idx, point in data.iterrows():
            cls_dist = self._tree_search(point, self._tree[-1])
            predicted_class = cls_dist.mode()['z'][0]
            label.append(predicted_class)
            probability = len(
                cls_dist[cls_dist['z'] == predicted_class])/len(cls_dist)
            prob.append(probability)
        return label, prob
    def _best_node(self, X_subset, y_subset):
        # calculate gini with received data
        gini_before = self._calc_gini(y_subset)
        print('gini before: ', gini_before)
        # select a feature and corresponding value
        features = X_subset.columns.values
        deltas = defaultdict(list)
        for feature in features:
            uniques = X_subset[feature].unique()
            for value in uniques:  
                # calculate left and right subset
                left_subset = y_subset.iloc[
                    X_subset.index[X_subset[feature] <= value]]
                right_subset = y_subset.iloc[
                    X_subset.index[X_subset[feature] > value]]
                # for each subset, calculate gini
                left_gini = self._calc_gini(left_subset)
                right_gini = self._calc_gini(right_subset)
                gini_after = (len(
                    left_subset)/len(X_subset))*left_gini
                + (len(right_subset)/len(X_subset))*right_gini
                # calculate delta, and make it default if max gain.
                delta = gini_before - gini_after
                # store the delta along with feature and value
                deltas[feature].append({value:[delta, gini_after]})
        # find the right feature-value combination which has maximum delta
        max_gain = 0
        node = None
        for name, feature in deltas.items():
            for value_delta_gini in feature:
                value, delta_gini = next(iter(value_delta_gini.items()))
                if delta_gini[0] > max_gain:
                    node = Node(
                        name=name, 
                        value=value, 
                        delta=delta_gini[0], 
                        gini_after=delta_gini[1])
                    max_gain = delta_gini[0]
        #print('best node: ', node)
        return node
            
    def _calc_gini(self, y_subset):
        target_name = y_subset.columns.values
        classes = y_subset[target_name[0]].unique()
        ratio_sum = 0
        for label in classes:
            num_instances = len(y_subset[y_subset[target_name[0]] == label])
            ratio_sum += ((num_instances/len(y_subset)) ** 2)
        gini = 1-ratio_sum
        return gini
                
    def _subset(self, node, X_subset, y_subset):
        """
        subsets the data into right and left subsets based on feature split.
        """
        left_subset_X = X_subset[
            X_subset[node.feature_name] <= node.feature_value]
        left_subset_y = y_subset.iloc[
            X_subset.index[X_subset[node.feature_name] <= node.feature_value]]
        right_subset_X = X_subset[
            X_subset[node.feature_name] > node.feature_value]
        right_subset_y = y_subset.iloc[
            X_subset.index[X_subset[node.feature_name] > node.feature_value]]
        return left_subset_X, left_subset_y, right_subset_X, right_subset_y
    @property
    def tree_(self):
        return self._tree

# Training

In [4]:
data = pd.read_csv('dummy.csv')
X_train = data[['x1', 'x2', 'x3']]
y_train = data[['z']]

In [5]:
model =  DecisionTree(depth=2, score='gini')
model.fit(X_train, y_train)

cur_depth:  0
gini before:  0.6577777777777778
feature_name:  x1 feature_split:  4.1 delta:  0.6577777777777778 gini:  0.0
cur_depth:  1
gini before:  0.0
this is a pure leaf
class distribution:     z
0  1
1  1
2  1
3  1
4  1
5  1
cur_depth:  1
gini before:  0.49382716049382713
feature_name:  x1 feature_split:  4.5 delta:  0.49382716049382713 gini:  0.0
cur_depth:  2
gini before:  0.0
this is a pure leaf
class distribution:     z
0  0
cur_depth:  2
gini before:  0.5
feature_name:  x1 feature_split:  5.5 delta:  0.5 gini:  0.0
class distribution:     z
0  2
class distribution:     z
0  0
1  2
2  0
3  2
4  2
5  0
6  0


## results for tree above

Root node: N1: X1 = 4.1<br>
Gini before split: 0.6577<br>
Gini after split: 0.292929<br>
Delta: 0.361<br>
Left Child: Leaf with distribution [1,1,1,1,1,1]<br>
Right Child: N2<br>

Node N2: X1 = 6.9<br>
Gini before split: 0.4938271<br>
Gini after split: 0.292929<br>
Delta: 0.1975<br>
Left Child: N3<br>
right Child: Leaf with distribution [0,0,0]<br>

Node N3: X1 = 4.5<br>
Gini before split: 0.44444<br>
Gini after split: 0.2666<br>
Delta: 0.1777<br>
Left child: Leaf with distribution [0]<br>
Right child: leaf with distribution [2,2,2,2,0]<br>

# Testing

In [6]:
test_data = []
test_data.append([4.1, -0.1, 2.2])
test_data.append([6.1, 0.4, 1.3])

In [7]:
test = pd.DataFrame(test_data, columns=['x1', 'x2', 'x3'])

In [8]:
label, probability = model.predict_proba(test)
for idx, point in enumerate(test_data):
    print('POINT: ', point, ' LABEL: ', label[idx], ' PROB: ', probability[idx])

POINT:  [4.1, -0.1, 2.2]  LABEL:  1  PROB:  1.0
POINT:  [6.1, 0.4, 1.3]  LABEL:  0  PROB:  0.5714285714285714
